# Lab 10

In [1]:
!#git clone -b poleval/task3 https://github.com/n-waves/fastai --depth 1 

#### Imports

In [2]:
from typing import Dict, List, Tuple, Optional
from collections import namedtuple
import os

In [3]:
from functional import seq, pseq
from functional.streams import Sequence
from IPython.core.display import HTML

def display_seq(sequence:Sequence,rows:int)-> None:
        sequence._repr_html_= lambda :sequence.tabulate(rows,tablefmt='html')
        display(sequence)
        sequence._repr_html_= lambda :sequence.tabulate(10,tablefmt='html')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
import functools
def compose(*functions):
    def compose2(f, g):
        return lambda x: f(g(x))
    return functools.reduce(compose2, functions, lambda x: x)

## Setup

In [6]:
!tree blobs.bak/work/up_low50k/

blobs.bak/work/up_low50k/
├── models
│   └── fwd_v50k_finetune_lm_enc.h5
└── tmp
    ├── sp-50k.model
    └── sp-50k.vocab

2 directories, 3 files


In [7]:
fastai_model_path = "./blobs.bak/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
sentencepiece_model_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.model"
sentencepiece_vocab_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.vocab"

In [8]:
def exists_or_ex(path):
    file = open(path,"r")
    file.close()
_ = [exists_or_ex(path) for path in [fastai_model_path, sentencepiece_model_path, sentencepiece_vocab_path]]

## Sentencepiece

In [122]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(sentencepiece_model_path)

spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")
# spm_processor.LoadVocabulary(sentencepiece_vocab_path,threshold= 1)

True

In [123]:
# spm_processor.encode_as_pieces("Ala ma kota i kot alę")


In [124]:
for n in range(5):
    print(spm_processor.SampleEncodeAsPieces('New York', -1, 0.1))

[b'<s>', b'\xe2\x96\x81N', b'e', b'w', b'\xe2\x96\x81York', b'</s>']
[b'<s>', b'\xe2\x96\x81New', b'\xe2\x96\x81Y', b'or', b'k', b'</s>']
[b'<s>', b'\xe2\x96\x81N', b'ew', b'\xe2\x96\x81Yo', b'rk', b'</s>']
[b'<s>', b'\xe2\x96\x81N', b'e', b'w', b'\xe2\x96\x81York', b'</s>']
[b'<s>', b'\xe2\x96\x81New', b'\xe2\x96\x81York', b'</s>']


In [125]:
from fastai.text import *

In [126]:
! head -n ./blobs.bak/

head: invalid number of lines: ‘./blobs.bak/’


In [127]:
# wgts = torch.load(fastai_model_path, map_location=lambda storage, loc: loc)
wgts = torch.load(fastai_model_path, map_location='cuda:0')
# enc_wgts = to_np(wgts['0.encoder.weight'])
# row_m = enc_wgts.mean(0)



In [109]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [110]:
# enc_wgts = to_np(wgts['encoder.weight']) #I think it expects  wgts top have 0.endoder.weight
# row_m = enc_wgts.mean(0)
torch.cuda.set_device(0)

UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

lm_path=  fastai_model_path

# em_sz,nh,nl = 400,1150,3  # that should be the case
em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],lm_path)
lm.reset()
lm.eval()


SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [111]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self): return len(self.x)



In [112]:
from torchtext import data,vocab

### From infer

In [176]:
# inpucik = "Doktor Marian Bubak sprawdzi za kilka dni nasze egzaminy, cieszymy się z tego powodu bo"
inpucik = "Mama bardzo dobrze"
test_ids = [np.array(spm_processor.encode_as_ids(inpucik))]
test_ids

[array([  2, 618,   8,  91, 431,   3])]

In [178]:
test_ds = LMTextDataset(test_ids)
test_samp = SortSampler(test_ids, key=lambda x: len(test_ids[x]))
test_dl = DataLoader(test_ds, bs, transpose=True, transpose_y=True, num_workers=1, pad_idx=PAD_ID, sampler=test_samp, pre_pad=False)
# x,y = list(test_dl)[0]
# x2 = [int(i) for i in x]
# y2 = [int(i) for i in y]
# spm_processor.decode_ids(x2)

'Mama bardzo dobrze'

In [313]:
def next_word(ss,model,toks_at_once):
    ids = [np.array(spm_processor.encode_as_ids(ss))]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(test_ids[x]))
    dl = DataLoader(test_ds, bs, transpose=True, transpose_y=True, num_workers=1, pad_idx=PAD_ID, sampler=test_samp, pre_pad=False)
    
    tensor1 = None
#     model.reset() # todo:bcm - do or dont'?
    with no_grad_context():
        for (x, y) in dl:
            tensor1= model(x)
    p= tensor1[0]
    r = torch.multinomial(p[-1].exp(), toks_at_once)
    print(r.size())
    predicted_ids = [int(r_) for r_ in r]
    return spm_processor.decode_ids(predicted_ids)

In [314]:
def next_words(ss,lm, n_words, toks_at_once=1):
    wip = ss
    for i  in range(n_words):
        wip =  wip + " "+ next_word(wip,lm,toks_at_once)
    return wip

In [315]:
next_words("Policjant ogląda",lm,1,19)

torch.Size([19])


'Policjant ogląda zbrodniwszy obwodow swoje szopk się przez swoim i popełni rozmow obraz mat wszystko dzik ogniska migawk Małysz na'

## Ten do pachu

In [285]:
def getp():
    dl = test_dl
    loss = 0.0
    tensor1 = None
    # lm.reset()
    with no_grad_context():
        for (x, y) in tqdm(dl):
            print("ala")
            targets = y.view(-1)
            preds = lm(x)[0]
            tensor1= lm(x)
            not_pads = targets != PAD_ID
            ce = F.cross_entropy(preds[not_pads], targets[not_pads], reduction='sum')
            loss += ce
    loss
    p= tensor1[0]
    return p

In [289]:
def get_prediction(n):
    p = getp()
    r = torch.multinomial(p[-1].exp(), n)
    predicted_ids = [int(r_) for r_ in r]
    return spm_processor.decode_ids(predicted_ids)

In [290]:
inpucik

'Mama bardzo dobrze'

In [291]:
get_prediction(20)

100%|██████████| 1/1 [00:00<00:00, 25.54it/s]


'komponuje zrozumiała bawił poznała chwal dba znał patrzy rozwija ogląda u zrobiła oddaje odnowił starała poznawa wypowie się kontynuuje powiedziała'

In [269]:
# lm.predict(text="ala ma kota", n_words=10, temperature=0.75) works in newer version

In [53]:
import fastai
from fastai import learner

In [54]:
learner.load_model(lm,"")

FileNotFoundError: [Errno 2] No such file or directory: ''

In [17]:
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))


drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]
learner.freeze_to(-1)

learner.model.load_state_dict(wgts)

lr=1e-2
lrs = lr

learner



NameError: name 'md' is not defined

# Worse

In [76]:
from fastai.torch_core import *
from fastai.basic_data import *
from fastai.callback import *
from fastai.data_block import *
from fastai.utils.ipython import gpu_mem_restore
import inspect
from fastprogress.fastprogress import format_time, IN_NOTEBOOK
from time import time
from fastai.sixel import plot_sixel

def load_that_learner(torch_thing,**db_kwargs):
    "Load a `Learner` object saved with `export_state` in `path/file` with empty data, optionally add `test` and load on `cpu`. `file` can be file-like (file or buffer)"
    state = torch_thing
    model = state.pop('model')
    src = LabelLists.load_state(path, state.pop('data'))
    if test is not None: src.add_test(test)
    data = src.databunch(**db_kwargs)
    cb_state = state.pop('cb_state')
    clas_func = state.pop('cls')
    res = clas_func(data, model, **state)
    res.callback_fns = state['callback_fns'] #to avoid duplicates
    res.callbacks = [load_callback(c,s, res) for c,s in cb_state.items()]
    return res

In [79]:
load_that_learner(wgts)

KeyError: 'model'

In [77]:
LM_PATH = "./blobs.bak/work/up_low50k/models"
LM_FILE = "fwd_v50k_finetune_lm_enc.h5"
model = load_learner(LM_PATH,LM_FILE,load_on='cpu')

RuntimeError: Attempting to deserialize object on CUDA device 1 but torch.cuda.device_count() is 1. Please use torch.load with map_location to map your storages to an existing device.

In [62]:
help(load_learner)

Help on function load_learner in module fastai.basic_train:

load_learner(path:Union[pathlib.Path, str], file:Union[pathlib.Path, str, _io.BufferedWriter, _io.BytesIO]='export.pkl', test:fastai.data_block.ItemList=None, **db_kwargs)
    Load a `Learner` object saved with `export_state` in `path/file` with empty data, optionally add `test` and load on `cpu`. `file` can be file-like (file or buffer)



In [28]:
help(get_language_model)

Help on function get_language_model in module fastai.text.learner:

get_language_model(arch:Callable, vocab_sz:int, config:dict=None, drop_mult:float=1.0)
    Create a language model from `arch` and its `config`, maybe `pretrained`.

